In [1]:
import os 
import json
import pandas as pd
import traceback
from langchain_together import Together
from langchain_core.callbacks.base import BaseCallbackHandler

# Load the .env file variable


In [2]:
class TokenUsageTracker(BaseCallbackHandler):
    def __init__(self):
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.total_cost = 0

    def on_llm_start(self, serialized, prompts, **kwargs):
        self.prompt_tokens += sum(len(prompt) for prompt in prompts)

    def on_llm_new_token(self, token, **kwargs):
        self.total_tokens += 1
        self.completion_tokens += 1

    def on_llm_end(self, response, **kwargs):
        self.total_cost = response.llm_output["cost"]

In [3]:
# Load the environment file (containing the api keys)
from dotenv import load_dotenv

load_dotenv() 

True

In [4]:
KEY = os.getenv("TOGETHER_API_KEY")

print(KEY)

60455415480d979c18ac478e5761e69ae3dfefc27c7eb7b11d7a85aa0d8f292c


In [5]:
# Using the langchain class to get llm
token_tracker = TokenUsageTracker()
callbacks = [token_tracker]

llm = Together(
    model="openchat/openchat-3.5-1210",
    top_k=1,
    together_api_key= KEY,
)

input_ = """You are a teacher with a deep knowledge of machine learning and AI. \
You provide succinct and accurate answers. Answer the following question: 

What is a large language model?"""
print(llm.invoke(input_))

/opt/anaconda3/envs/mk-llm/lib/python3.10/site-packages/langchain_together/llms.py:94: UserWarning: The completions endpoint, has 'max_tokens' as required argument. The default value is being set to 200 Consider setting this value, when initializing LLM
  warnings.warn(




A large language model is a type of artificial intelligence (AI) system that uses deep learning techniques to generate human-like text. These models are trained on massive amounts of text data, allowing them to understand and generate text in a way that is similar to how humans process language. Large language models can be used for a variety of applications, such as natural language processing, machine translation, and text summarization. Some popular large language models include OpenAI's GPT-3 and Google's BERT.

These models are capable of generating highly coherent and contextually relevant text, making them valuable tools for a wide range of tasks. However, they can also produce text that is nonsensical or biased, so it is important to use them responsibly and with appropriate safeguards in place. Overall, large language models represent a significant advancement in the field of AI and have the potential to revolutionize many aspects of human communication and interaction.


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [8]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker, Given the above text, it is your job to \
create a quiz pf {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}"""

In [9]:
# Creating prompt template
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number","subject", "tone", "response_json"],
    template= TEMPLATE
)

In [10]:
# LLMChain: Chain to run queries against the llm
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

In [11]:
TEMPLATE2 ="""
You are an expert english grammarian and writer. Given a multiple choice quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis
if the quiz is not at par with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [13]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
# Connecting two chain together, sequential chain

generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz","review"], verbose=True)

In [15]:
file_path = "../data.txt"

In [16]:
file_path

'../data.txt'

In [17]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

Artificial intelligence

Machine learning as subfield of AI[22]
As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[23] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[24]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[24]: 488  By 1980, expert systems had come to dominate AI, and statistics was out of favor.[25] Work on symbolic/knowledge-ba

In [19]:
# Serialize the python dictionary into a JSON- formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE ="Normal"

In [21]:
# Setting up token usage tracking 

response=generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    })


/opt/anaconda3/envs/mk-llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Artificial intelligence

Machine learning as subfield of AI[22]
As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[23] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[24]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[24]: 488  By 198

In [22]:
print(response)



{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}


In [ ]:
print(f"Total tokens: {cb.total_tokens}")
print(f"Prompt token: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
response

In [ ]:
quiz = response.get("quiz")

In [ ]:
json.loads(quiz)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options ="|".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv", index=False)